<a href="https://colab.research.google.com/github/pranshurastogi29/how-effective-are-std-drugs/blob/master/STD_AI_Entity_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, norm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import warnings
warnings.filterwarnings("ignore")

import os
import gc
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [0]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

def create_model(data, catcols):    
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = 180
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)

    inp1 = layers.Input(shape=(2,))
    x = layers.Dense(2, activation="sigmoid")(inp)
    outputs.append(x)
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="sigmoid")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="sigmoid")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=[inputs,inp1], outputs=y)
    return model



In [0]:
df = pd.read_csv('/content/drive/My Drive/train.csv')
df1 = pd.read_csv('/content/drive/My Drive/test.csv')
df = df.drop(columns=['drug_approved_by_UIC','patient_id'])
df1 = df1.drop(columns=['drug_approved_by_UIC','patient_id'])
df1['base_score'] = -1
df.head()

,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,number_of_times_prescribed,base_score
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27,8.022969
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192,7.858458
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17,6.341969
3,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37,6.590176
4,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,43,6.144782


In [0]:
from sklearn.preprocessing import  PowerTransformer,QuantileTransformer
rng = np.random.RandomState(24546)
qt =  PowerTransformer()
X = df.number_of_times_prescribed.values.reshape(-1,1)
#X = qt.fit_transform(X)
qt1 = QuantileTransformer(n_quantiles=500, output_distribution='normal',
                         random_state=rng)
X = qt.fit_transform(X)
print('this is done')
df['number_of_times_prescribed'] = pd.Series(X.reshape(1,-1)[0])
X = df1.number_of_times_prescribed.values.reshape(-1,1)
#X = qt.fit_transform(X)
qt1 = QuantileTransformer(n_quantiles=500, output_distribution='normal',
                         random_state=rng)
X = qt.fit_transform(X)
print('this is done')
df1['number_of_times_prescribed'] = pd.Series(X.reshape(1,-1)[0])

this is done
this is done


In [0]:
from sklearn.preprocessing import  PowerTransformer,QuantileTransformer
rng = np.random.RandomState(24546)
qt =  PowerTransformer()
X = df.effectiveness_rating.values.reshape(-1,1)
X = qt.fit_transform(X)
qt1 = QuantileTransformer(n_quantiles=500, output_distribution='normal',
                         random_state=rng)
X = qt1.fit_transform(X)
print('this is done')
df['effectiveness_rating'] = pd.Series(X.reshape(1,-1)[0])
X = df1.effectiveness_rating.values.reshape(-1,1)
X = qt.fit_transform(X)
qt1 = QuantileTransformer(n_quantiles=500, output_distribution='normal',
                         random_state=rng)
X = qt1.fit_transform(X)
print('this is done')
df1['effectiveness_rating'] = pd.Series(X.reshape(1,-1)[0])

this is done
this is done


In [0]:
from sklearn.preprocessing import  PowerTransformer,QuantileTransformer
rng = np.random.RandomState(24546)
qt =  PowerTransformer()
X = df.base_score.values.reshape(-1,1)
#X = qt.fit_transform(X)
qt = QuantileTransformer(n_quantiles=500, output_distribution='normal',
                         random_state=rng)
X = qt.fit_transform(X)
print('this is done')
df['base_score'] = pd.Series(X.reshape(1,-1)[0])

this is done


In [0]:
df1

,name_of_drug,review_by_patient,number_of_times_prescribed,use_case_for_drug,effectiveness_rating,base_score
0,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",0.271432,Depression,5.199338,-1
1,Contrave,"""Contrave combines drugs that were used for al...",0.680894,Weight Loss,0.256461,-1
2,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",-0.159650,Keratosis,-0.717694,-1
3,Methadone,"""Ive been on Methadone for over ten years and ...",0.231879,Opiate Withdrawal,-0.324591,-1
4,Ambien,"""Ditto on rebound sleepless when discontinued....",0.892369,Insomnia,-1.013919,-1
...,...,...,...,...,...,...
10755,Chantix,"""I took chantix a little over a month. It made...",-1.358755,Smoking Cessation,-5.199338,-1
10756,Armodafinil,"""This medicine kept me from sleeping the whole...",0.102778,Narcolepsy,-5.199338,-1
10757,Tamoxifen,"""I have taken Tamoxifen for 5 years. Side effe...",0.870839,"Breast Cancer, Prevention",5.199338,-1
10758,Levonorgestrel,"""I&#039;m married, 34 years old and I have no ...",-0.618579,Birth Control,-0.105686,-1


In [0]:
df

,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,number_of_times_prescribed,base_score
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",0.243504,0.451782,0.757254
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",-0.118323,2.360086,0.632244
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",-0.602120,0.063449,-0.366994
3,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",0.243504,0.730172,-0.127849
4,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",-1.013919,0.866803,-0.585413
...,...,...,...,...,...,...
32160,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now...",0.243504,1.562625,0.108212
32161,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",-0.474322,-2.101707,-5.199338
32162,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting...",0.243504,0.385470,-0.460314
32163,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now....",-0.118323,0.276815,0.725118


In [0]:
df1['base_score'] = -1
train = df[['name_of_drug', 'use_case_for_drug','effectiveness_rating','number_of_times_prescribed','base_score']]
test = df1[['name_of_drug', 'use_case_for_drug','effectiveness_rating','number_of_times_prescribed','base_score']]

In [0]:
test

,name_of_drug,use_case_for_drug,effectiveness_rating,number_of_times_prescribed,base_score
0,Mirtazapine,Depression,5.199338,0.271432,-1
1,Contrave,Weight Loss,0.256461,0.680894,-1
2,Zyclara,Keratosis,-0.717694,-0.159650,-1
3,Methadone,Opiate Withdrawal,-0.324591,0.231879,-1
4,Ambien,Insomnia,-1.013919,0.892369,-1
...,...,...,...,...,...
10755,Chantix,Smoking Cessation,-5.199338,-1.358755,-1
10756,Armodafinil,Narcolepsy,-5.199338,0.102778,-1
10757,Tamoxifen,"Breast Cancer, Prevention",5.199338,0.870839,-1
10758,Levonorgestrel,Birth Control,-0.105686,-0.618579,-1


In [0]:
train

,name_of_drug,use_case_for_drug,effectiveness_rating,number_of_times_prescribed,base_score
0,Valsartan,Left Ventricular Dysfunction,0.243504,0.451782,0.757254
1,Guanfacine,ADHD,-0.118323,2.360086,0.632244
2,Lybrel,Birth Control,-0.602120,0.063449,-0.366994
3,Buprenorphine / naloxone,Opiate Dependence,0.243504,0.730172,-0.127849
4,Cialis,Benign Prostatic Hyperplasia,-1.013919,0.866803,-0.585413
...,...,...,...,...,...
32160,Cymbalta,Anxiety,0.243504,1.562625,0.108212
32161,Nexplanon,Birth Control,-0.474322,-2.101707,-5.199338
32162,Venlafaxine,Panic Disorde,0.243504,0.385470,-0.460314
32163,Fluoxetine,Obsessive Compulsive Disorde,-0.118323,0.276815,0.725118


In [0]:
data = pd.concat([train, test]).reset_index(drop=True)
features = [x for x in train.columns if x not in ['effectiveness_rating','number_of_times_prescribed','base_score']]

out = []
for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    data[feat] = lbl_enc.fit_transform(data[feat].fillna(-1).astype(str).values)
    out.append(lbc_enc)
import pickle
saved_embeddings_fname = "/content/drive/My Drive/labels.pickle"
with open(saved_embeddings_fname, 'wb') as f:
    pickle.dump(out, f, -1)

In [0]:
data.name_of_drug.dtype

dtype('int64')

In [0]:
features = ['name_of_drug', 'use_case_for_drug']

In [0]:
train = data[data.base_score != -1].reset_index(drop=True)
test = data[data.base_score == -1].reset_index(drop=True)
test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]

In [0]:
oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))
fold = 0
skf = KFold(n_splits=10)
for train_index, test_index in skf.split(train, train.base_score.values):
    print('fold',fold)
    X_train, X_test = train.iloc[train_index, :], train.iloc[test_index, :]
    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train, y_test = X_train.base_score.values, X_test.base_score.values
    model = create_model(data, features)
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])
    X_t = [X_train.loc[:, features].values[:, k] for k in range(X_train.loc[:, features].values.shape[1])]
    X_tt =  [X_test.loc[:, features].values[:, k] for k in range(X_test.loc[:, features].values.shape[1])]
    
    es = callbacks.EarlyStopping(monitor='mae', min_delta=0.0001, patience=5,
                                 verbose=1, mode='min', baseline=None, restore_best_weights=True)

    rlr = callbacks.ReduceLROnPlateau(monitor='mae', factor=0.5,
                                      patience=3, min_lr=1e-7, mode='min', verbose=1)
    
    x_t = X_train[['effectiveness_rating','number_of_times_prescribed']].values
    x_tt = X_test[['effectiveness_rating','number_of_times_prescribed']].values
    model.fit([X_t,x_t],
              y_train,
              validation_data = ([X_tt,x_tt],y_test),
              verbose=1,
              batch_size=1024,
              callbacks=[es,rlr],
              epochs=100
             )
    valid_fold_preds = model.predict([X_tt,x_tt])
    test_fold_preds = model.predict([test_data,
                        test[['effectiveness_rating','number_of_times_prescribed']].values])
    oof_preds[test_index] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    fold = fold + 1
    K.clear_session()


fold 0
Epoch 1/100
29/29 [==============================] - 1s 18ms/step - loss: 1.3088 - mae: 0.9031 - val_loss: 1.1548 - val_mae: 0.8324 - lr: 0.0010
Epoch 2/100
29/29 [==============================] - 0s 11ms/step - loss: 1.1910 - mae: 0.8603 - val_loss: 1.1145 - val_mae: 0.8164 - lr: 0.0010
Epoch 3/100
29/29 [==============================] - 0s 11ms/step - loss: 1.1237 - mae: 0.8330 - val_loss: 1.0819 - val_mae: 0.8025 - lr: 0.0010
Epoch 4/100
29/29 [==============================] - 0s 11ms/step - loss: 1.0780 - mae: 0.8163 - val_loss: 1.0744 - val_mae: 0.7989 - lr: 0.0010
Epoch 5/100
29/29 [==============================] - 0s 11ms/step - loss: 1.0522 - mae: 0.8050 - val_loss: 1.0710 - val_mae: 0.7971 - lr: 0.0010
Epoch 6/100
29/29 [==============================] - 0s 11ms/step - loss: 1.0382 - mae: 0.7978 - val_loss: 1.0697 - val_mae: 0.7963 - lr: 0.0010
Epoch 7/100
29/29 [==============================] - 0s 11ms/step - loss: 1.0292 - mae: 0.7940 - val_loss: 1.0689 - val_mae

In [0]:
use_case_for_drug = model.get_layer('use_case_for_drug').get_weights()[0]
name_of_drug = model.get_layer('name_of_drug').get_weights()[0]

In [0]:
len(data.use_case_for_drug.unique())

670

In [0]:
import pickle
saved_embeddings_fname = "/content/drive/My Drive/embeddings.pickle"
with open(saved_embeddings_fname, 'wb') as f:
    pickle.dump([use_case_for_drug,name_of_drug], f, -1)

In [0]:
test_preds/10

array([0.02912089, 0.02415028, 0.13070462, ..., 0.00073672, 0.0058581 ,
       0.11667352])

In [0]:
test_preds = qt.inverse_transform(test_preds.reshape(-1,1))

In [0]:
test_preds.ravel()

array([7.21451693, 7.17099722, 8.59389419, ..., 6.80935728, 6.90164144,
       8.49878707])

In [0]:
sub = pd.read_csv('/content/drive/My Drive/test.csv')

In [0]:
sub['base_score'] = pd.Series(test_preds.ravel())

In [0]:
sub[['patient_id','base_score']].to_csv('stdai1.csv',index=False)